**Project Description**

**Vehicle Sales Data Analysis Project Overview:**

This notebook analyzes vehicle data to help customers make informed buying and selling decisions. The process transforms raw data into meaningful insights through systematic steps: data loading, exploration, cleaning, and visualization. Missing values are filled based on similar vehicle characteristics, data types are optimized, and all changes are validated. The final dataset reveals key patterns in vehicle prices, mileage, and features through clear visualizations. This structured approach ensures reliable data that helps understand what drives vehicle value in the market.

**SECTION 1: Data Import and Setup**

* Library Imports and Initialization

In [42]:
import os
import pandas as pd
import plotly.express as px
import altair as alt
import streamlit as st

* Dataset loading and Error Handling

In [ ]:
try:
    us_vehicles = pd.read_csv('../vehicles_us.csv')
except FileNotFoundError:
    print("CSV file not found")

**SECTION 2: Initial Data Assessment**

* DataFrame Properties and Types

In [44]:
display("Dataset info:")
display(us_vehicles.info())

'Dataset info:'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


None

* Dataset Structure and Preview

The us_vehicles dataset consists of 51,525 rows and 13 columns: price, model_year, model, condition, cylinders, fuel, odometer, transmission,  type, paint_color, is_4wd, date_posted, and days_listed. 
    
Data types include:
- int64: price, days_listed
- float64: model_year, cylinders, odometer, is_4wd
- object: model, condition, fuel, transmission, type, paint_color, date_posted

Columns containing missing values include:
- model_year, cylinders, odometer, is_4wd and paint_color

In [45]:
display("\nFirst Few Rows of Data:")
display(us_vehicles.head(10))

'\nFirst Few Rows of Data:'

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


**SECTION 3: Data Cleaning**

* After using .info() and .head() to see our data overview, we have found columns with missing data.

* More Specifically, is_4wd has the most missing values (nearly half), further investigation is needed to determine whether missing values indicate "Not 4WD" or truly unknown data.

In [46]:
display(us_vehicles['is_4wd'].value_counts(dropna=False))

is_4wd
NaN    25953
1.0    25572
Name: count, dtype: int64

Numeric Value Processing

The fill_missing_values function identifies and fill missing data using:
1. Median values by vehicle type for cylinders
2. Meddian values by model for model years
3. Meddian values by model for odmoeter readings

* Numeric Value Processing

In [47]:
def fill_missing_values(us_vehicles):
    us_vehicles['cylinders'] = us_vehicles[['cylinders', 'type']].groupby('type').transform(lambda x: x.fillna(x.median()))
    us_vehicles['model_year'] = us_vehicles[['model_year', 'model']].groupby('model').transform(lambda x: x.fillna(x.median()))
    us_vehicles['model_year'] = us_vehicles['model_year'].round().astype('int64')
    us_vehicles['odometer'] = us_vehicles[['odometer', 'model_year']].groupby('model_year').transform(lambda x: x.fillna(x.median()))
    us_vehicles['odometer'] = us_vehicles['odometer'].fillna(us_vehicles['odometer'].median())

    missing_values_count = us_vehicles.isnull().sum()
    print("Column with Missing Values:")
    for col in missing_values_count[missing_values_count > 0].index:
        display(f"**{col}**: {missing_values_count[col]} nulls")

fill_missing_values(us_vehicles)

Column with Missing Values:


'**paint_color**: 9267 nulls'

'**is_4wd**: 25953 nulls'

* Integer Conversion

In [48]:
for col in ['model_year', 'cylinders']:
    decimal_flags = us_vehicles[col].dropna().apply(lambda x: x % 1 !=0)
    if any(decimal_flags):
        display(f"{col} has decimal flags")
    else:
        display(f"{col} can be safely converted to int64 or bool, if applicable")

'model_year can be safely converted to int64 or bool, if applicable'

'cylinders can be safely converted to int64 or bool, if applicable'

* Integer Conversion and Verification for Model Year and Cylinders, and Boolean Conversion for 4WD Status

* Model Year and Cylinders are checked for integer conversion
* Is 4WD will be converted to a catergorical type to represent missing values as unknown instead of 0
* Replacing missing values (NaN) with 0 assumes that all represent "Not 4WD", which is currently unverified 

In [49]:
us_vehicles[['model_year', 'cylinders']] = us_vehicles[['model_year', 'cylinders']].astype('Int64')

us_vehicles['is_4wd'] = us_vehicles['is_4wd'].map({1.0: "Yes"}).fillna("Unknown").astype('category')
display(us_vehicles[['model_year', 'cylinders', 'is_4wd']].dtypes)

model_year       Int64
cylinders        Int64
is_4wd        category
dtype: object

#'model_year' fillna: Fill missing values in the 'model_year'column with 0 and convert it to int64 for consistency. Then, display the value counts after filling the blanks. The same approach is applied to columns 'cylinders', odometer, and 'is_4wd' for consistency.

In [50]:
#us_vehicles['model_year'] = us_vehicles['model_year'].fillna(0).astype(int)
#display(us_vehicles['model_year'].value_counts())

#'cylinders' fillna: Code modified for 'cylinders' column

In [51]:
#us_vehicles['cylinders'] = us_vehicles['cylinders'].fillna(0).astype(int)
#display(us_vehicles['cylinders'].value_counts())

#'odometer' fillna: Code modified for 'odometer' column

In [52]:
#us_vehicles['odometer'] = us_vehicles['odometer'].fillna(0).astype(int)
#display(us_vehicles['odometer'].value_counts())

#'is_4wd' fillna: Code modified for 'is_4wd' column

In [53]:
#us_vehicles['is_4wd'] = us_vehicles['is_4wd'].fillna(0).astype(int)
#display(us_vehicles['is_4wd'].value_counts())

* Paint Color Processing and Standardization

* Paint color data is converted to category data type, with missing values marked as "unknown"

In [54]:
us_vehicles['paint_color'] = us_vehicles['paint_color'].str.lower().str.strip()
display(us_vehicles['paint_color'].value_counts())

us_vehicles['paint_color'] = us_vehicles['paint_color'].fillna('unknown')
display(us_vehicles['paint_color'].value_counts())

us_vehicles['paint_color'] = us_vehicles['paint_color'].astype('str').astype('category') 
display(us_vehicles['paint_color'].dtype)

paint_color
white     10029
black      7692
silver     6244
grey       5037
blue       4475
red        4421
green      1396
brown      1223
custom     1153
yellow      255
orange      231
purple      102
Name: count, dtype: int64

paint_color
white      10029
unknown     9267
black       7692
silver      6244
grey        5037
blue        4475
red         4421
green       1396
brown       1223
custom      1153
yellow       255
orange       231
purple       102
Name: count, dtype: int64

CategoricalDtype(categories=['black', 'blue', 'brown', 'custom', 'green', 'grey',
                  'orange', 'purple', 'red', 'silver', 'unknown', 'white',
                  'yellow'],
, ordered=False, categories_dtype=object)

Recheck cleaned first Few Rows

In [55]:
display("\nFirst Few Rows of Cleaned Data:")
display(us_vehicles.head(10))

'\nFirst Few Rows of Cleaned Data:'

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6,gas,145000.0,automatic,SUV,unknown,Yes,2018-06-23,19
1,25500,2011,ford f-150,good,6,gas,88705.0,automatic,pickup,white,Yes,2018-10-19,50
2,5500,2013,hyundai sonata,like new,4,gas,110000.0,automatic,sedan,red,Unknown,2019-02-07,79
3,1500,2003,ford f-150,fair,8,gas,161397.0,automatic,pickup,unknown,Unknown,2019-03-22,9
4,14900,2017,chrysler 200,excellent,4,gas,80903.0,automatic,sedan,black,Unknown,2019-04-02,28
5,14990,2014,chrysler 300,excellent,6,gas,57954.0,automatic,sedan,black,Yes,2018-06-20,15
6,12990,2015,toyota camry,excellent,4,gas,79212.0,automatic,sedan,white,Unknown,2018-12-27,73
7,15990,2013,honda pilot,excellent,6,gas,109473.0,automatic,SUV,black,Yes,2019-01-07,68
8,11500,2012,kia sorento,excellent,4,gas,104174.0,automatic,SUV,unknown,Yes,2018-07-16,19
9,9200,2008,honda pilot,excellent,6,gas,147191.0,automatic,SUV,blue,Yes,2019-02-15,17


**SECTION 4: Adding or Reviewing Additional Factors**

* Missing Data Analysis

In [56]:
display("Missing Dataset info:")
display(us_vehicles.isnull().sum())

'Missing Dataset info:'

price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

* Vehicles Type Analysis

In [57]:
type_counts = us_vehicles['type'].value_counts()
display(type_counts)

type
SUV            12405
truck          12353
sedan          12154
pickup          6988
coupe           2303
wagon           1541
mini-van        1161
hatchback       1047
van              633
convertible      446
other            256
offroad          214
bus               24
Name: count, dtype: int64

* Statisical Analysis

In [58]:
display("Statistical Summary:")
display(us_vehicles.describe())

'Statistical Summary:'

,price,model_year,cylinders,odometer,days_listed
count,51525.000000,51525.0,51525.0,51525.000000,51525.00000
mean,12132.464920,2009.793557,6.13081,115199.286453,39.55476
std,10040.803015,6.099381,1.658414,62082.376299,28.20427
min,1.000000,1908.0,3.0,0.000000,0.00000
25%,5000.000000,2007.0,4.0,73500.000000,19.00000
50%,9000.000000,2011.0,6.0,114074.000000,33.00000
75%,16839.000000,2014.0,8.0,152827.000000,53.00000
max,375000.000000,2019.0,12.0,990000.000000,271.00000


Checking high price vehicle counts

In [60]:
high_price_vehicles = us_vehicles[us_vehicles['price'] >= 100000]

high_price_model_counts = high_price_vehicles.groupby('model')['price'].count().reset_index()

high_price_model_counts.columns = ['model', 'count']

display(high_price_model_counts.sample(min(20, len(high_price_model_counts))))

,model,count
5,gmc sierra 2500hd,1
3,ford f-150,6
0,chevrolet corvette,1
7,ram 2500,2
1,chevrolet silverado 1500,1
4,ford f-250 super duty,1
6,nissan frontier,1
8,toyota tundra,1
2,chevrolet suburban,3


#Select only the important vehicle features needed for our analysis (price, year, model, condition, etc.)
Remove all other columns to keep our data focused and easier to work with

In [ ]:
#columns_to_use = ['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel', 'odometer', 'transmission', 'type', 'paint_color']
#us_vehicles = us_vehicles[columns_to_use]

Recheck Statisical Analysis Review

In [ ]:
#display("Statistical Summary:")
#display(us_vehicles.describe())

**SECTION 5: Data Visualization**

* Key Visual Attribute Relationships

This section creates four charts showing:
1. Price patterns across vehicle types
+ Shows how many vehicles fall into different price ranges
+ Separates vehicles by type (like cars, trucks, SUVs)
+ Makes it easy to see what prices are most common for each type
2. How mileage affects price
+ Scatter plot shows how prices decrease with higher mileage
+ Vehicle types are color-coded, with paint colors analyzed separately for oxidation risk (high/moderate/low)
3. Compare different vehicle features against each other
+ Looks at fuel type, mileage, transmission, and number of cylinders
+ Colors show price differences
+ Helps find relationships, specifically whether cars with more cylinders use more fuel
4. Compare price and mileage across fuel types
+ Compare costs and value retention across electric, hybrid and gas vehicles based on mileage


In [ ]:
def analyze_vehicle_charts(us_vehicles):

    # PRICE DISTRIBUTION HISTOGRAM
    hist_price_by_type = px.histogram(us_vehicles, x='price', color='type', nbins=30,
                    title='Vehicle Price Distribution by Type')
    st.plotly_chart(hist_price_by_type)

    # PRICE vs MILEAGE SCATTER PLOT
    scatter_price_vs_mileage = px.scatter(us_vehicles, x='odometer', y='price', color='type',
                    title='Mileage Impact on Price Across Vehicle Types')
    st.plotly_chart(scatter_price_vs_mileage)

    # TECHNICAL ATTRIBUTES SCATTER MATRIX
    matrix_technical_features = px.scatter_matrix(us_vehicles,
        dimensions=['fuel', 'odometer', 'transmission', 'cylinders'],
        color='price')
    st.plotly_chart(matrix_technical_features)
    
    # PRICE vs MILEAGE BY FUEL TYPE
    fig_fuel_comparison = px.scatter(
        us_vehicles, x='odometer', y='price', color='fuel',
        title='Price vs Mileage by Fuel Type',
        labels={'odometer': 'Mileage (miles)', 'price': 'Price $', 'fuel': 'Fuel Type'}
    )
    st.plotly_chart(fig_fuel_comparison)


    return hist_price_by_type, scatter_price_vs_mileage, matrix_technical_features, fig_fuel_comparison

